In [1]:
#Import all packages 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import random
from sklearn import metrics
import sklearn
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer


import warnings
warnings.filterwarnings("ignore")
from __future__ import division

In [2]:
import os
import torch
from torch import nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
X = torch.rand((5,6))
#X = np.random.rand(5,6)
y = torch.tensor([-1, 1, -1, 1, 1])
#y = np.array([-1, 1, -1, 1, 1])
w = torch.rand(X.shape[1], requires_grad = True)
b = 0.5 #threshold

In [4]:
x_train = np.load("F1_2000_x_train.npy")

x_test = np.load("F1_2000_x_test.npy")

y_train = np.load("F1_2000_y_train.npy")

y_test = np.load("F1_2000_y_test.npy")

print("Stratified selection of class labels:{} ".format(np.unique(y_train, return_counts=True)))


mm = make_pipeline(MinMaxScaler(), Normalizer())
x_train = mm.fit_transform(x_train)
x_test = mm.transform(x_test)

ones_train = np.ones(x_train.shape[0])
ones_test = np.ones(x_test.shape[0])

ones_train.shape = (x_train.shape[0],1)
ones_test.shape = (x_test.shape[0],1)

x_train = np.append(x_train, ones_train, axis = 1)
x_test = np.append(x_test, ones_test, axis = 1)

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

#w = torch.rand(x_train.shape[1], requires_grad = True)

w = torch.tensor(torch.FloatTensor(x_train.shape[1]).uniform_(-1, 1), requires_grad = True)

print(w)

b = 0.5 #threshold


Stratified selection of class labels:(array([-1,  1], dtype=int32), array([ 856, 1144])) 
tensor([ 0.9819, -0.4815,  0.6332,  0.7133,  0.6134, -0.4286,  0.5506,
        -0.9163, -0.3983, -0.8378,  0.9037, -0.8973,  0.5829,  0.1902,
        -0.6596,  0.2781,  0.4921,  0.8717,  0.3093,  0.8141,  0.5505,
         0.3575, -0.8456, -0.7688,  0.9862, -0.0005, -0.0179,  0.6251,
        -0.2774, -0.6065,  0.8751,  0.7457,  0.6839,  0.8273, -0.7167,
        -0.9226, -0.1978,  0.0525, -0.1350, -0.4951,  0.7814,  0.9631,
         0.8441,  0.8857,  0.3829,  0.9864, -0.0120,  0.1877, -0.5585,
        -0.7464, -0.2764,  0.3727, -0.7605, -0.4680,  0.2375,  0.7660])


In [5]:
data_pos = []
data_neg = []

for i in range(y_train.shape[0]):    
    if(y_train[i] == 1):
        data_pos.append(i)
    elif(y_train[i] == -1):
        data_neg.append(i)

In [6]:
y_pos_len = len(data_pos)
y_neg_len = len(data_neg)

In [7]:
print(y_pos_len)
print(y_neg_len)

1144
856


In [ ]:
def sum_loss(X, y, w, b):
    sum_l = 0
    for i in range(X.shape[0]):
        #sum_l = sum_l + max(0, 1-y[i].double()*(torch.sign(torch.dot(w,X[i])) - b))
        #sum_l = sum_l + max(0, 1-y[i].double()*(torch.dot(w,X[i]).double() - b))
        sum_l = sum_l + max(0, 1-y[i]*(torch.dot(w,X[i]) - b))
    return sum_l

In [ ]:
sum_loss(x_train.float(),y_train.float(),w,b)

In [ ]:
learning_rate = 0.01
num_epoch = 100

loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)

langrange_lambda = torch.tensor(0.01, dtype=torch.float, requires_grad=True)
psi = torch.tensor([y_pos_len - loss_pos], requires_grad= True).double()

In [ ]:
for i in range(num_epoch):
    
    print(f"Inside {i}")

    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    #print(f"Before backword: {w.grad}")
    
    #print(f"Before backword: {psi.grad}")
    
    w.retain_grad()
    

    final_loss = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss.backward(retain_graph=True)
    
    
    w.retain_grad()


    #print(f"After Backword: {w.grad}")
    #print(f"After Backword: {psi.grad}")
    
    w = w - learning_rate * w.grad
    
    
    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    
    psi.retain_grad()
    
    final_loss_psi = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double()+ ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss_psi.backward(retain_graph=True)
    
    psi.retain_grad()
    
    psi = psi - learning_rate * psi.grad
    
    #print(f"After update: {w.grad}")
    #print(f"After update: {psi.grad}")
    
    
    #w.grad.zero_()
    #psi.grad.zero_()
    #langrange_lambda.grad.zero_()

    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    
    langrange_lambda.retain_grad()

    final_loss_a = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss_a.backward(retain_graph=True)
    
    langrange_lambda.retain_grad()
    #(final_loss_a).backward()
    langrange_lambda = langrange_lambda + learning_rate * langrange_lambda.grad
    
    loss_after_update = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    
    print(f"Loss value at {i}th epoch is: {loss_after_update}")

    #w.grad.zero_()
    #psi.grad.zero_()
    #langrange_lambda.grad.zero_()


In [ ]:
print(w)
print(psi)
print(langrange_lambda)

In [ ]:
print(w.grad)
print(langrange_lambda.grad)
print(psi.grad)

In [ ]:
for i in range(num_epoch):
    
    print(f"Inside {i}")
    
    
    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    
    langrange_lambda.retain_grad()

    final_loss_a = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss_a.backward(retain_graph=True)
    
    langrange_lambda.retain_grad()
    #(final_loss_a).backward()
    langrange_lambda = langrange_lambda + learning_rate * langrange_lambda.grad
    

    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    #print(f"Before backword: {w.grad}")
    
    #print(f"Before backword: {psi.grad}")
    
    w.retain_grad()
    

    final_loss = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss.backward(retain_graph=True)
    
    
    w.retain_grad()


    #print(f"After Backword: {w.grad}")
    #print(f"After Backword: {psi.grad}")
    
    w = w - learning_rate * w.grad
    
    
    loss_pos = sum_loss(x_train[data_pos].float(), y_train[data_pos].float(), w, b)
    loss_neg = sum_loss(x_train[data_neg].float(), y_train[data_neg].float(), w, b)
    
    
    psi.retain_grad()
    
    final_loss_psi = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double()+ ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    final_loss_psi.backward(retain_graph=True)
    
    psi.retain_grad()
    
    psi = psi - learning_rate * psi.grad
    
    #print(f"After update: {w.grad}")
    #print(f"After update: {psi.grad}")
    
    
    #w.grad.zero_()
    #psi.grad.zero_()
    #langrange_lambda.grad.zero_()
    
    loss_after_update = (1.0/psi) * torch.tensor(loss_neg).double() + langrange_lambda.double() * torch.tensor(loss_pos).double() + ((1.0/psi) - langrange_lambda.double()) * y_pos_len + psi * langrange_lambda.double()
    
    print(f"Loss value at {i}th epoch is: {loss_after_update}")

    #w.grad.zero_()
    #psi.grad.zero_()
    #langrange_lambda.grad.zero_()


In [ ]:
print(w)
print(psi)
print(langrange_lambda)

In [ ]:
x_test

In [ ]:
def f1_score_pr(precision, recall):
        
    denom = precision+ recall

    if(denom == 0):
        denom = 1
    f1 = 2 * (precision * recall) / (denom)

    return f1

In [ ]:
y_hat = np.dot(x_test, w.detach().numpy())
y_hat[y_hat > 0] = 1
y_hat[y_hat < 0] = -1
print("Prediction vector y_hat: {}".format(np.unique(y_hat, return_counts= True)))
precision = metrics.precision_score(y_test, y_hat)
recall = metrics.recall_score(y_test, y_hat)
f1 = f1_score_pr(precision, recall)
print("f1 score : {}".format(f1))